# Multiple Water Surface Profiles with Specific Energy and Hydraulic Jump Analysis

This notebook lets you simulate multiple flow profiles upstream of a weir, and calculates specific energy and checks for hydraulic jump conditions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import ipywidgets as widgets
from IPython.display import display

g = 9.81  # gravitational acceleration (m/s^2)

In [ ]:
# --- Functions ---
def compute_critical_depth(Q, b):
    return (Q**2 / (g * b**2))**(1/3)

def compute_normal_depth(Q, b, n, S0):
    func = lambda y: (1/n) * b * y**(5/3) * np.sqrt(S0) - Q
    y_guess = (Q**2 / (g * b**2))**(1/3) * 1.5
    return fsolve(func, y_guess)[0]

def Sf(y, Q, b, n):
    A = b * y
    R = y
    return (n**2 * Q**2) / (A**2 * R**(4/3))

def Fr(Q, b, y):
    return Q / (b * y * np.sqrt(g * y))

def specific_energy(y, Q, b):
    v = Q / (b * y)
    return y + v**2 / (2*g)

def sequent_depth(y1, Q, b):
    Fr1 = Fr(Q, b, y1)
    if Fr1 <= 1:
        return None  # No jump expected
    return 0.5 * y1 * (-1 + np.sqrt(1 + 8*Fr1**2))

def plot_profiles(Q_list, b, n, S0, L, dx, weir_type, weir_height, C_b, C_d):
    plt.figure(figsize=(12, 6))
    
    for Q in Q_list:
        y_crit = compute_critical_depth(Q, b)
        y_normal = compute_normal_depth(Q, b, n, S0)
        y_start = y_crit * 1.05

        x_vals = [0]
        y_vals = [y_start]
        x, y = 0, y_start

        while x < L:
            fr = Fr(Q, b, y)
            denom = 1 - fr**2
            if abs(denom) < 1e-6:
                break
            dy_dx = (S0 - Sf(y, Q, b, n)) / denom
            y += dy_dx * dx
            x += dx
            x_vals.append(x)
            y_vals.append(y)

        label = f"Q={Q} m³/s"
        plt.plot(x_vals, y_vals, label=label)

        E_start = specific_energy(y_start, Q, b)
        y2 = sequent_depth(y_start, Q, b)

        print(f"=== Flow Rate Q={Q} m³/s ===")
        print(f"Critical Depth: {y_crit:.3f} m")
        print(f"Normal Depth: {y_normal:.3f} m")
        print(f"Specific Energy at Start: {E_start:.3f} m")
        if y2:
            print(f"Sequent Depth (Hydraulic Jump Expected): {y2:.3f} m")
        else:
            print("No Hydraulic Jump (Flow Subcritical)")
        print("\n")

    plt.axhline(y=compute_critical_depth(Q_list[0], b), color='r', linestyle='--', label='Critical Depth')
    plt.title(f'Multiple Backwater Profiles ({weir_type})')
    plt.xlabel('Distance upstream (m)')
    plt.ylabel('Water Depth (m)')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
flow_selector = widgets.SelectMultiple(
    options=[5, 10, 15, 20, 25],
    value=[10],
    description='Flow Rates Q (m³/s)',
)

ui = widgets.VBox([
    flow_selector,
    widgets.FloatSlider(value=3.0, min=1.0, max=10.0, step=0.5, description='Channel Width b (m)'),
    widgets.FloatSlider(value=0.03, min=0.01, max=0.1, step=0.005, description="Manning's n"),
    widgets.FloatLogSlider(value=0.001, base=10, min=-4, max=-1, step=0.1, description='Slope S₀'),
    widgets.FloatSlider(value=100.0, min=10, max=300, step=10, description='Length L (m)'),
    widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='dx Step (m)'),
    widgets.Dropdown(options=['broad-crested', 'sharp-crested'], value='broad-crested', description='Weir Type'),
    widgets.FloatSlider(value=1.0, min=0.5, max=3.0, step=0.1, description='Weir Height (m)'),
    widgets.FloatSlider(value=1.7, min=1.0, max=2.5, step=0.1, description='C_b (broad)'),
    widgets.FloatSlider(value=0.6, min=0.4, max=0.9, step=0.05, description='C_d (sharp)'),
])

out = widgets.interactive_output(
    plot_profiles,
    {
        'Q_list': flow_selector,
        'b': ui.children[1],
        'n': ui.children[2],
        'S0': ui.children[3],
        'L': ui.children[4],
        'dx': ui.children[5],
        'weir_type': ui.children[6],
        'weir_height': ui.children[7],
        'C_b': ui.children[8],
        'C_d': ui.children[9]
    }
)

display(ui, out)